In [ ]:
## Environment Setup

This workshop now uses **uv** for modern Python dependency management. 

### Quick Setup
If you haven't set up the environment yet:

```bash
# Install dependencies
uv sync

# Activate environment (optional - uv run handles this automatically)
source .venv/bin/activate

# Start Jupyter
uv run jupyter lab
```

### Verify Installation
Run the cell below to verify all packages are installed correctly.

In [ ]:
# Verify environment setup
try:
    import openai
    import azure.ai.projects
    import azure.ai.inference
    import azure.identity
    import pandas as pd
    import numpy as np
    
    print("✅ Environment Setup Successful!")
    print("-" * 40)
    print(f"📦 OpenAI SDK: {openai.__version__}")
    print(f"📦 Pandas: {pd.__version__}")
    print(f"📦 NumPy: {np.__version__}")
    print("\n🚀 Ready to proceed with the workshop!")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("\n💡 Setup required:")
    print("1. Run: uv sync")
    print("2. Make sure you're using the correct kernel")
    print("3. Restart this notebook")